In [4]:
from PyPDF2 import PdfReader
import re,os
import pandas as pd
import numpy as np

In [5]:
#This is not good but works


In [6]:
for file in os.listdir('GabaritosLaranjaPDF'):
    enem = PdfReader('GabaritosLaranjaPDF/'+file)
    try:
        text = enem.pages[0].extract_text()
        text = re.sub('\xa0',' ',text)
        text = re.sub(' +',' ',text)
        is_english = 'inglês' in text.lower()
        gab = [i.split(' ') for i in re.findall(r'[\d]+ [A-E](?: [A-E])?(?!nulad|NULAD)',text)]
        #treat prova de espanhol:
        new_gab = []
        for answ in gab:
            if len(answ)>2:
                new_gab.append(np.array([answ[0],answ[2],'spanish']))
                new_gab.append(np.array([answ[0],answ[1],'english']))
            else:
                new_gab.append(np.array(answ+[None]))
        gab = new_gab
        gab = np.stack(gab)
        df = pd.DataFrame({'question':gab[:,0],'answer':gab[:,1],'test':gab[:,2]})
        df['question'] = pd.to_numeric(df['question'])
        df=df.sort_values('question').reset_index(drop=True)
        min_quest = 1 if df['question'].min() < 90 else 91
        max_quest = min_quest+90
        
    except Exception as e:
        print(e)
    if (len(df)!=90 and len(df)!=95) and not 'anulad' in text.lower():
        raise Exception('Enem has 90 questions')
    elif (len(df)!=90 or len(df)!=95):
        for question in range(min_quest,max_quest):
            if question not in df['question'].values:
                df.loc[len(df)] = [question,'anulada',None]
        if is_english:
            for question in range(min_quest,min_quest+5):
                if len(df[df['question']==question]) < 2:
                    if 'english' not in df[df['question']==question]['question'].tolist():
                        df.loc[len(df)] = [question,'anulada','english']
                    if 'spanish' not in df[df['question']==question]['question'].tolist():
                        df.loc[len(df)] = [question,'anulada','spanish']
    df.sort_values('question').to_csv('GabaritosCsv/'+file.strip('.pdf')+'.csv',index=False)

In [5]:
df

,question,answer,test
0,91,E,english
1,91,C,spanish
2,92,C,spanish
3,92,D,english
4,93,B,spanish
...,...,...,...
90,176,E,None
91,177,A,None
92,178,C,None
93,179,B,None
